| Experiment Name | Description | Parameters |
| :- | :- | :- |
| EXP01 |   Reference run as EXP00                                      | rnDt      = 2700.                 |
| EXP02 |   Decreased time step for stability                           | rnDt      = 900.                  |    
| EXP03 |   Decreased wind forcing                                      | zTau0     = 5e-2                  |
| EXP04 |   (Almost) Centered around equator                            | rnPhi0    = -40                   | 
| EXP05 |   Uniform zonal wind forcing                                  | nnforcing = 2                     |
| EXP06 |   TEOS10                                                      | nnforcing = 2, TEOS10             |
| EXP07 |   No flattening of the bathymetry                             | ln_equ_flat = False               |
| EXP08 |   Centered wind forcing with minumum on meridional boundary   | ln_equ_flat = True, rn_Phi=-21    |
| EXP09 |   As EXP08 but with flattening at the equator                 | ln_equ_flat = False               |

In [ ]:
import xarray as xr
from matplotlib import pyplot as plt
import cftime._cftime as cft
#import holoviews as hv

#hv.extension('bokeh')

In [ ]:
def clean_coords(xrdataset, start_year):
    try:
        shift = cft.timedelta(days=360 * start_year)
        datetimeindex = (xrdataset.indexes['time_counter'] + shift).to_datetimeindex
    except:
        datetimeindex = xrdataset.indexes['time_counter']
    try:
        latitude  = xrdataset.nav_lat[:,0].values
        longitude = xrdataset.nav_lon[0,:].values
    except:
        latitude  = xrdataset.nav_lat_grid_T[:,0].values
        longitude = xrdataset.nav_lon_grid_T[0,:].values

    return xrdataset.assign_coords(dict({'time_counter':datetimeindex, 'y': latitude, 'x': longitude}))

In [ ]:
def read_abort(path, dict_vars={'votemper': 'T', 'vosaline': 'S', 'vozocrtx': 'U', 'vomecrty': 'V', 'vovecrtz': 'W', 'sossheig': 'H', 'vovvldep': 'depth'}):
    abort = xr.load_dataset(path)
    abort = abort.rename(dict_vars)
    abort = abort.set_coords(['nav_lon', 'nav_lat', 'depth'])
    abort = abort.assign_coords({'nav_lev': abort.nav_lev * -1})
    abort = abort.assign_coords({'depth': abort.depth * -1})
    return(abort[list(dict_vars.values())])


# **EXP01: Reference Run**

In [ ]:
U1       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP01/BASIN_grid_U.nc")
T1       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP01/BASIN_grid_T.nc")
mask1    = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP01/mesh_mask.nc")

In [ ]:
T1 = clean_coords(T1, 2000)
U1 = clean_coords(U1, 2000)
mask1 = clean_coords(mask1, 2000)

## Bathymetry

In [ ]:
mask1.e3t_0.sum(dim='nav_lev').plot(x='x', y='y')

In [ ]:
mask1.x

In [ ]:
(- mask1.e3t_0.isel(y=20)).sum(dim='nav_lev').plot()

## Salinity

In [ ]:
T1.soce.isel(deptht=0, time_counter=-2).plot(x='nav_lon_grid_T', y='nav_lat_grid_T', vmin=33, vmax=37)

## Zonal velocity

In [ ]:
U1.uoce.isel(depthu=0, time_counter=-2).plot(x='nav_lon', y='nav_lat')


# **EXP02: Decreased time-step, 900s**

In [ ]:
U2       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP02/BASIN_grid_U.nc")
T2       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP02/BASIN_grid_T.nc")

In [ ]:
T2 = clean_coords(T2, 2000)
U2 = clean_coords(U2, 2000)

## Salinity at the surface

In [ ]:
T2.soce.isel(deptht=0, time_counter=-5).plot(x='nav_lon_grid_T', y='nav_lat_grid_T', vmin=33, vmax=37)

## Zonal velocity at the surface

In [ ]:
U2.uoce.isel(depthu=0, time_counter=-5).plot(x='nav_lon', y='nav_lat')


# **EXP03: Decreased wind forcing x0.5**

In [ ]:
U3       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP03/BASIN_grid_U.nc")
T3       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP03/BASIN_grid_T.nc")

In [ ]:
T3 = clean_coords(T3, 2000)
U3 = clean_coords(U3, 2000)

## Salinity at the surface

In [ ]:
T3.soce.isel(deptht=0, time_counter=-2).plot(x='nav_lon_grid_T', y='nav_lat_grid_T', vmin=33, vmax=37)

## Zonal velocity at the surface

In [ ]:
U3.uoce.isel(depthu=0, time_counter=-3).plot(x='nav_lon', y='nav_lat')


# **EXP04: Include southern hemisphere**

In [ ]:
U4       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP04/BASIN_grid_U.nc")
T4       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP04/BASIN_grid_T.nc")
mask4    = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP04/mesh_mask.nc")

In [ ]:
T4 = clean_coords(T4, 2000)
U4 = clean_coords(U4, 2000)
mask4 = clean_coords(mask4, 2000)

## Bathymetry

In [ ]:
mask4.e3t_0.sum(dim='nav_lev').plot(x='x', y='y')

## Salinity at the surface

In [ ]:
T4.soce.isel(deptht=0, time_counter=-1).plot(x='nav_lon_grid_T', y='nav_lat_grid_T', vmin=33, vmax=37)

## Zonal velocity at the surface

In [ ]:
U4.uoce.isel(depthu=3, time_counter=-1).plot(x='nav_lon', y='nav_lat')


# **EXP05: Uniform zonal wind forcing --> 247**

In [ ]:
U5       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP05/BASIN_grid_U.nc")
T5       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP05/BASIN_grid_T.nc")

In [ ]:
T5 = clean_coords(T5, 2000)
U5 = clean_coords(U5, 2000)

## Zonal velocity at the surface

In [ ]:
U5.uoce.isel(depthu=0, time_counter=-1).plot(x='nav_lon', y='nav_lat')

## Salinity at the surface

In [ ]:
T5.soce.isel(deptht=-5, time_counter=-1).plot(x='nav_lon_grid_T', y='nav_lat_grid_T', vmin=33, vmax=40)


# **EXP06: Change to TEOS10**

In [ ]:
U6       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP06/BASIN_grid_U.nc")
T6       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP06/BASIN_grid_T.nc")

In [ ]:
T6 = clean_coords(T6, 2000)
U6 = clean_coords(U6, 2000)

## Salinity at the surface

In [ ]:
T6.soce.isel(deptht=5, time_counter=-1).plot(x='nav_lon_grid_T', y='nav_lat_grid_T', vmin=33, vmax=37)

## Zonal velocity at the surface

In [ ]:
U6.uoce.isel(depthu=0, time_counter=-1).plot(x='nav_lon', y='nav_lat', cmap='viridis')


# **EXP07: No flattening of the bathymetry at the equator**

In [ ]:
U7       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP07/BASIN_grid_U.nc")
T7       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP07/BASIN_grid_T.nc")
mask7    = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP07/mesh_mask.nc")

In [ ]:
T7 = clean_coords(T7, 2000)
U7 = clean_coords(U7, 2000)
mask7 = clean_coords(mask7, 2000)

## Bathymetry

In [ ]:
mask7.e3t_0.sum(dim='nav_lev').plot(x='x', y='y')

In [ ]:
mask7

## Salinity at the surface

In [ ]:
T7.soce.isel(deptht=-2, time_counter=-1).plot(x='nav_lon_grid_T', y='nav_lat_grid_T', vmin=33, vmax=37)

## Zonal velocity at the surface

In [ ]:
U7.uoce.isel(depthu=0, time_counter=-1).plot(x='nav_lon', y='nav_lat')


# **EXP08: Wolfe & Cassi with minimum at the meridional boundaries**

In [ ]:
U8       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP08/BASIN_grid_U.nc")
T8       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP08/BASIN_grid_T.nc")
V8       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP08/BASIN_grid_V.nc")
abort8   = read_abort("/home/dkamm/NEMOGCM/tests/BASIN2/EXP08/output.abort.nc")

T8 = clean_coords(T8, 2000)
U8 = clean_coords(U8, 2000)
V8 = clean_coords(V8, 2000)

## Wind forcing

In [ ]:
U8.utau.isel(time_counter=1).plot()


## Zonal velocity at the surface

In [ ]:
U8.uoce.isel(depthu=5, time_counter=-1).plot(x='nav_lon', y='nav_lat')

In [ ]:
abort8.S.isel(y=21).plot( y='depth', x='nav_lon')

In [ ]:
abort8.U.isel(y=21).plot( y='depth', x='nav_lon', vmax=0.001)

# **EXP09: As EXP08, but without flattened topography at the equator**

In [ ]:
U9       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP09/BASIN_grid_U.nc")
T9       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP09/BASIN_grid_T.nc")
V9       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP09/BASIN_grid_V.nc")
abort9   = read_abort("/home/dkamm/NEMOGCM/tests/BASIN2/EXP09/output.abort.nc")
mask9    = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP09/mesh_mask.nc")

T9 = clean_coords(T9, 2000)
U9 = clean_coords(U9, 2000)
V9 = clean_coords(V9, 2000)

In [ ]:
abort9.S.isel(y=21).plot( y='depth', x='nav_lon')

In [ ]:
abort9.U.isel(y=21).plot( y='depth', x='nav_lon', vmax=0.002)

In [ ]:
dom   = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP08/domain_cfg_out.nc")

In [ ]:
dom.e3t.isel(time_counter=0).plot()

In [ ]:
dom.gphit.plot()